In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
datatrx = pd.read_csv('../input/rfm-moody/UAS_RFM', sep=';', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152,2021-05-06,"521.814,00"
1,1000158,2021-03-20,"722.981,00"
2,1000156,2021-06-30,"331.792,00"
3,1000115,2021-10-02,"136.963,00"
4,1000164,2021-02-28,"824.386,00"


In [3]:
datatrx.head(15)

,CardID,TrxDate,Amount
0,1000152,2021-05-06,"521.814,00"
1,1000158,2021-03-20,"722.981,00"
2,1000156,2021-06-30,"331.792,00"
3,1000115,2021-10-02,"136.963,00"
4,1000164,2021-02-28,"824.386,00"
5,1000136,2021-04-26,"880.497,00"
6,1000147,2021-06-24,"557.250,00"
7,1000118,2021-01-19,"420.393,00"
8,1000167,2021-05-02,"525.623,00"
9,1000160,2021-04-19,"110.583,00"


In [4]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.8+ KB


In [5]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-01 00:00:00 2021-12-06 00:00:00


In [6]:
sd = dt.datetime(2021,12,18)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(15)

,CardID,TrxDate,Amount,hist
0,1000152,2021-05-06,"521.814,00",226.0
1,1000158,2021-03-20,"722.981,00",273.0
2,1000156,2021-06-30,"331.792,00",171.0
3,1000115,2021-10-02,"136.963,00",77.0
4,1000164,2021-02-28,"824.386,00",293.0
5,1000136,2021-04-26,"880.497,00",236.0
6,1000147,2021-06-24,"557.250,00",177.0
7,1000118,2021-01-19,"420.393,00",333.0
8,1000167,2021-05-02,"525.623,00",230.0
9,1000160,2021-04-19,"110.583,00",243.0


In [7]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    object        
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 7.8+ KB


In [8]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(15)

,recency,frequency,monetary_value
CardID,,,
1000101,299.0,1,"176.615,00"
1000102,78.0,3,"52.251,00749.596,00673.070,00"
1000103,42.0,1,"369.132,00"
1000104,105.0,2,"666.368,00862.348,00"
1000105,204.0,5,"669.384,00944.422,00819.506,00556.868,00723.52..."
1000106,179.0,2,"741.032,00455.039,00"
1000107,135.0,3,"359.375,00543.371,00529.418,00"
1000108,182.0,3,"559.297,00233.883,00967.279,00"
1000109,232.0,2,"143.884,00688.258,00"


In [9]:
quartiles = rfmTable.quantile(q=[0.20,0.40,0.60])
print(quartiles, type(quartiles))

     recency  frequency
0.2     45.0        1.0
0.4    138.0        2.0
0.6    184.0        3.0 <class 'pandas.core.frame.DataFrame'>


In [11]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 4
    elif x <= d[p][0.40]:
        return 3
    elif x <= d[p][0.60]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    else:
        return 4

In [14]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))

In [17]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \

rfmSeg.head(15)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,RFMClass
CardID,,,,,,
1000101,299.0,1,"176.615,00",1,1,11
1000102,78.0,3,"52.251,00749.596,00673.070,00",3,3,33
1000103,42.0,1,"369.132,00",4,1,41
1000104,105.0,2,"666.368,00862.348,00",3,2,32
1000105,204.0,5,"669.384,00944.422,00819.506,00556.868,00723.52...",1,4,14
1000106,179.0,2,"741.032,00455.039,00",2,2,22
1000107,135.0,3,"359.375,00543.371,00529.418,00",3,3,33
1000108,182.0,3,"559.297,00233.883,00967.279,00",2,3,23
1000109,232.0,2,"143.884,00688.258,00",1,2,12


In [18]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Berlangganan: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Pemborong: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Beresiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Nyaris Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Kehilangan Pelanggan Hemat: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan Tetap:  0
Berlangganan:  16


KeyError: 'M_Quartile'